# Pipeline for processing Vietnamese text

In [1]:
!pip install underthesea pyvi unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.4 MB/s eta 0:00:00


In [ ]:
import re
from underthesea import pos_tag, word_tokenize
from pyvi import ViTokenizer
from unidecode import unidecode

## Preprocess input text

In [ ]:
def preprocess(text):
    text = re.sub(r"[^a-zA-ZÀ-ỹà-ỹ\s]", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = text.strip()
    return text

## Handle diacritic

In [ ]:
def remove_diacritics(text):
    return unidecode(text)

## Tokenization with many options

In [ ]:
def tokenize(text, method='underthesea'):
    if method == 'underthesea':
        return word_tokenize(text)
    elif method == 'pyvi':
        return ViTokenizer.tokenize(text).split()
    elif method == 'naive':
        return text.split()
    else:
        raise ValueError("Unsupported tokenization method")

## POS tagging

In [ ]:
def pos_tagging(tokens, method='underthesea'):
    if method == 'underthesea':
        return pos_tag(' '.join(tokens))
    else:
        raise ValueError("Only underthesea supported for POS tagging now")

## Full pipeline

In [ ]:
def run_pipeline(text):
    print("Original:", text)
    preprocessed = preprocess(text)
    print("Preprocessed:", preprocessed)
    normalized = remove_diacritics(preprocessed)
    print("Diacritics:", normalized)
    tokens = tokenize(preprocessed)
    print("Tokens:", tokens)
    pos_tags = pos_tag(preprocessed)
    print("POS Tags:", pos_tags)
    print("-" * 50)

## Test

In [ ]:
test_sentences = [
    "Xin chào! Đây là một ví dụ đơn giản. Bạn có khỏe không?",
    "Tôi thích ăn phở, bún bò Huế, và cả mì Quảng nữa!!!",
    "Công nghệ AI đang phát triển rất nhanh; đặc biệt là trong lĩnh vực y tế & giáo dục.",
    "‘Tôi đã nói với anh ta: “Đừng làm vậy nữa!”,’ cô ấy kể lại.",
    "Dữ liệu đến từ nhiều nguồn: báo chí, mạng xã hội, và thậm chí cả tin đồn.",
    "Hệ thống này hoạt động tốt ở cả thành thị lẫn nông thôn - điều này rất quan trọng."
]

for sentence in test_sentences:
    run_pipeline(sentence)

Original: Xin chào! Đây là một ví dụ đơn giản. Bạn có khỏe không?
Preprocessed: Xin chào Đây là một ví dụ đơn giản Bạn có khỏe không
Diacritics: Xin chao Day la mot vi du don gian Ban co khoe khong
Tokens: ['Xin', 'chào Đây', 'là', 'một', 'ví dụ', 'đơn giản', 'Bạn', 'có', 'khỏe', 'không']
POS Tags: [('Xin', 'V'), ('chào Đây', 'V'), ('là', 'V'), ('một', 'M'), ('ví dụ', 'N'), ('đơn giản', 'A'), ('Bạn', 'N'), ('có', 'V'), ('khỏe', 'N'), ('không', 'R')]
--------------------------------------------------
Original: Tôi thích ăn phở, bún bò Huế, và cả mì Quảng nữa!!!
Preprocessed: Tôi thích ăn phở bún bò Huế và cả mì Quảng nữa
Diacritics: Toi thich an pho bun bo Hue va ca mi Quang nua
Tokens: ['Tôi', 'thích', 'ăn', 'phở', 'bún bò', 'Huế', 'và', 'cả', 'mì', 'Quảng nữa']
POS Tags: [('Tôi', 'P'), ('thích', 'V'), ('ăn', 'V'), ('phở', 'N'), ('bún bò', 'N'), ('Huế', 'Np'), ('và', 'C'), ('cả', 'T'), ('mì', 'N'), ('Quảng nữa', 'N')]
--------------------------------------------------
Original: Công ng

# Pipeline for conversion from Vietnamese to Sign language

# Preprocessing

In [12]:
import re
from underthesea import sent_tokenize

def preprocess(text):
    # Remove special characters (except Vietnamese accents and common punctuation)
    text = re.sub(r'[^\w\sÀ-ỹ,.!?]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Example
text = "Tôi   đã ăn   cơm   rồi!!! Bạn thì sao???"
output = preprocess(text)
print(output)

Tôi đã ăn cơm rồi!!! Bạn thì sao???


## Remove stopwords

In [13]:
from pyvi import ViTokenizer
import os

stopwords = set()
with open("/content/vietnamese-stopwords.txt", encoding='utf-8') as f:
    for line in f:
        stopwords.add(line.strip())

emotion_words = {'vui', 'buồn', 'tức giận', 'lo lắng', 'nhé', 'đấy'} # Need to expand to more emotion words

def remove_stopwords(sentence):
    words = ViTokenizer.tokenize(sentence).split()
    content_words = [w for w in words if w.replace('_', ' ') not in stopwords]
    emotion_tags = [w for w in words if w.replace('_', ' ') in emotion_words]
    return content_words, emotion_tags

# Example
filtered, emotion = remove_stopwords(output)
print("Filtered:", filtered)
print("Emotion Words:", emotion)

Filtered: ['Tôi', 'cơm', '!', '!', '!', 'Bạn', '?', '?', '?']
Emotion Words: []


## Change words order TODO

## Tokenization + POS tagging

In [5]:
from underthesea import pos_tag, word_tokenize

def tokenize_and_pos_tag(text):
    tokens = word_tokenize(text, format="text")
    pos_tags = pos_tag(tokens)
    return pos_tags

example_text = "Đây là một ví dụ về việc phân tích từ loại trong tiếng Việt."
result = tokenize_and_pos_tag(example_text)
print(result)

[('Đây', 'P'), ('là', 'V'), ('một', 'M'), ('ví_dụ', 'N'), ('về', 'E'), ('việc', 'N'), ('phân_tích', 'V'), ('từ_loại', 'N'), ('trong', 'E'), ('tiếng', 'N'), ('Việt', 'Np'), ('.', 'CH')]


## Convert words to gloss

In [20]:
gloss_dict = {
    "ôi_trời_ơi": "OH_MY_GOD",
    "hôm_nay": "TODAY",
    "tôi": "I",
    "quá": "VERY",
    "mệt_mỏi": "TIRED",
    "vì": "BECAUSE",
    "phải": "HAVE_TO",
    "làm_việc": "WORK",
    "đến": "UNTIL",
    "tận": "VERY",
    "khuya": "LATE_NIGHT"
}

def convert_to_gloss(pos_tags):
    gloss_sentence = []
    for word, pos in pos_tags:
        gloss = gloss_dict.get(word, word.upper())
        gloss_sentence.append(gloss)
    return gloss_sentence

## Add facial expression tags

In [21]:
def add_facial_expression(gloss_sentence):
    final_output = []
    for word in gloss_sentence:
        # Emotion-based expression
        if word in ["OH_MY_GOD"]:
            final_output.append("[SURPRISED EXPRESSION]")
        elif word in ["VERY"]:
            final_output.append("[INTENSITY FACIAL MARKER]")
        elif word in ["TIRED"]:
            final_output.append("[SAD EXPRESSION]")
        elif word in ["BECAUSE"]:
            final_output.append("[REASONING EXPRESSION]")
        final_output.append(word)
    return final_output

## Generate output

In [14]:
def generate_output(text):
    print("Original:", text)
    text = preprocess(text)
    print("Preprocessed:", text)

    tagged = tokenize_and_pos_tag(text)
    print("POS Tagged:", tagged)

    gloss = convert_to_gloss(tagged)
    print("Gloss:", gloss)

    gloss_with_expr = add_facial_expression(gloss)
    print("Final with facial expressions:", gloss_with_expr)

    return ' '.join(gloss_with_expr)

In [22]:
example_text = "Ôi trời ơi! Hôm nay tôi quá mệt mỏi vì phải làm việc đến tận khuya."

print("\nGenerated Gloss Output:\n")
result = generate_output(example_text)
print("\nFinal Output Text:\n", result)


Generated Gloss Output:

Original: Ôi trời ơi! Hôm nay tôi quá mệt mỏi vì phải làm việc đến tận khuya.
Preprocessed: Ôi trời ơi! Hôm nay tôi quá mệt mỏi vì phải làm việc đến tận khuya.
POS Tagged: [('Ôi', 'I'), ('trời_ơi', 'N'), ('! Hôm_nay', 'N'), ('tôi', 'P'), ('quá', 'R'), ('mệt_mỏi', 'A'), ('vì', 'E'), ('phải', 'V'), ('làm_việc', 'V'), ('đến', 'E'), ('tận', 'T'), ('khuya', 'A'), ('.', 'CH')]
Gloss: ['ÔI', 'TRỜI_ƠI', '! HÔM_NAY', 'I', 'VERY', 'TIRED', 'BECAUSE', 'HAVE_TO', 'WORK', 'UNTIL', 'VERY', 'LATE_NIGHT', '.']
Final with facial expressions: ['ÔI', 'TRỜI_ƠI', '! HÔM_NAY', 'I', '[INTENSITY FACIAL MARKER]', 'VERY', '[SAD EXPRESSION]', 'TIRED', '[REASONING EXPRESSION]', 'BECAUSE', 'HAVE_TO', 'WORK', 'UNTIL', '[INTENSITY FACIAL MARKER]', 'VERY', 'LATE_NIGHT', '.']

Final Output Text:
 ÔI TRỜI_ƠI ! HÔM_NAY I [INTENSITY FACIAL MARKER] VERY [SAD EXPRESSION] TIRED [REASONING EXPRESSION] BECAUSE HAVE_TO WORK UNTIL [INTENSITY FACIAL MARKER] VERY LATE_NIGHT .
